## CS431/631 Big Data Infrastructure
### Winter 2020 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Yan Shi_
* **ID:** _20809455_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [1]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize

# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    distinct_token = set()                   # Set that stores distinct tokens
    distinct_tokenpairs = set()              # Set that stores distinct token pairs
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        for token in t:
            distinct_token.add(token)
            for i in range(t.index(token)+1, len(t)):
                if t[i] != token:             # Not consider reflexive token pairs
                    distinct_tokenpairs.add((token, t[i]))
                    distinct_tokenpairs.add((t[i], token))     # count(x,y) and (y,x) both
                    
print("The number of distinct token is ", len(distinct_token))
print("The number of distinct token pairs is ", len(distinct_tokenpairs))
                

# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

The number of distinct token is  25975
The number of distinct token pairs is  1969760


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


In [8]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

filename = 'Shakespeare.txt'

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
def PMI_analysis_two(token_pairs, filename='Shakespeare.txt'):            # function for two-token queries PMI analysis
    count_x = 0                               # count the time that the first token occurs
    count_y = 0                               # count the time that the second token occurs
    count_xy = 0                              # count the time that the token pair occurs
    count_line = 0                            # count the time of lines in that file
    with open(filename) as f:
        for line in f:
            count_line += 1
            t = simple_tokenize(line)
            if (token_pairs[0] in t) and (token_pairs[1] in t):        # If the token pair occurs
                count_x += 1
                count_y += 1
                count_xy += 1
            elif token_pairs[0] in t:                                  # If only the first token occurs
                count_x += 1
            elif token_pairs[1] in t:                                  # If only the second token occurs
                count_y += 1
            else:
                pass
    p_x = count_x / count_line                                        # Calculate the p(x), p(y) and p(x,y)
    p_y = count_y / count_line
    p_xy = count_xy / count_line
    pmi = log(p_xy / (p_x * p_y), 10)                                 # Calculate the PMI, base is 10 
    return count_xy, pmi

def PMI_analysis_one(token, threshold, filename='Shakespeare.txt'):              # function for one-token queries PMI analysis
    token_pairs_dict = {}                       # Dictionary for token-pair : times the token-pair occurs
    pmi_dict = {}                               # Dictionary for token-pair : PMI of that token-pair
    count_x = 0                                 # count the time that token occurs
    with open(filename) as f:
        for line in f:
            t = simple_tokenize(line)
            if token in t:                     # If the token occurs in that line
                count_x += 1
                for eachitem in set(t):             # Construct token pairs which contain the objective token, first item of the tuple is 
                    if ((token, eachitem) in token_pairs_dict.keys()) and (token != eachitem):               # the objective token
                        token_pairs_dict[(token, eachitem)] += 1         # If the token have occurred before
                    elif token != eachitem:
                        token_pairs_dict[(token, eachitem)] = 1          # If the token have never occurred before
                    else:
                        pass
                    
    token_pairs_dict = dict(filter(lambda x: x[1]>=threshold, token_pairs_dict.items())) # Get the token-pairs occurring at least threshold
                                                                                         # times
    for pairs in token_pairs_dict.keys():               # compute the PMI of the token pairs selected in the last step
        _, pmi = PMI_analysis_two(pairs, filename)      # and store in a dictionary
        pmi_dict[pairs] = pmi
    pmi_dict = {k: v for k,v in sorted(pmi_dict.items(), key=lambda x: x[1], reverse=True)}
    pmi_dict = {i[0]: i[1] for i in list(pmi_dict.items())[:min(5, len(pmi_dict))]}      # select at most 5 token pairs(may not have 5 pairs)
                                                                                         # that have the highest PMI 
    return count_x, pmi_dict, token_pairs_dict
                
###################################################################################################################

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue

        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        count_1, pmi_1_dict, token_1_dict = PMI_analysis_one(q_tokens[0], threshold)     # Do the one-token queries 
        x = q_tokens[0]                                 # the objective token
        p = list(pmi_1_dict.items())                    # List for (token-pair, PMI of that token-pair)
        
        print("  n(%s) = %d"%(q_tokens[0], count_1))
        print("  high PMI tokens with respect to %s (threshold: %d):"%(x, threshold))
        for i in range(min(5, len(pmi_1_dict))):
            print("    n(%s, %s) = %d,  PMI(%s,%s) = %3f"%(x, p[i][0][1], token_1_dict[(x, p[i][0][1])], x, p[i][0][1], p[i][1]))
        # in the above, all XXX values should be at least as large as the threshold

    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        count_xy, pmi = PMI_analysis_two(q_tokens)
        print("  n(%s,%s) = %d"%(q_tokens[0], q_tokens[1], count_xy))
        print("  PMI(%s,%s) = %3f"%(q_tokens[0], q_tokens[1], pmi))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): follow
Input a positive integer frequency threshold: 100
  n(follow) = 298
  high PMI tokens with respect to follow (threshold: 100):
Input 1 or 2 space-separated tokens (return to quit): follow
Input a positive integer frequency threshold: 60
  n(follow) = 298
  high PMI tokens with respect to follow (threshold: 60):
    n(follow, and) = 90,  PMI(follow,and) = 0.177008
    n(follow, to) = 66,  PMI(follow,to) = 0.172361
    n(follow, the) = 62,  PMI(follow,the) = 0.020556
Input 1 or 2 space-separated tokens (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:

*It will be slower. For the two-token queries, there is only one for loop involved, the number of for loop being executed is exactly the number of lines in the file. Therefore, for example, the file becomes 100 times larger (the number of lines is 100 times more), the time need is 100 times more, when it exceeds the maximum times that for loop can execute, it fails to run.*

*The one-token queries will also be slower, the first for loop in that program is also directly related to the number of lines in the file (similar analysis to above), while other for loops are not directly related to the number of lines. Also, since the file is larger, the dictionary and list should also be larger, then they need more time to sort and filter. If the file become 100 times larger, the time needed will be way more than 100 times, it fails to run before the two token queries, maybe due to too many loop count or memory insufficient.*


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.